In [16]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json
  
import json

with open("/tmp/sarcasm.json", 'r') as f:
    data = json.load(f)
sent=[]
is_sar=[]
url=[]

for head in data:
  sent.append(head['headline'])
  is_sar.append(head['is_sarcastic'])
  url.append(head['article_link'])

print(sent[10])  

--2020-09-03 19:03:29--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 173.194.210.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.04s   

2020-09-03 19:03:29 (139 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]

airline passengers tackle man who rushes cockpit in bomb threat


In [46]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
t=Tokenizer(oov_token='<OOV>')
t.fit_on_texts(sent)
word_index=t.word_index
seq=t.texts_to_sequences(sent)
pad=pad_sequences(seq,padding='post')

print(len(pad[10]))

40


In [30]:
vocab_size=10000
embedding_dim=16
max_length=32
trunc_type='post'
oov_tok='<OOV>'
training_size=20000

training_sent=sent[0:training_size]
test_sent=sent[training_size:]
training_label=is_sar[0:training_size]
test_label=is_sar[training_size:]
print('Training Size: ',len(training_sent)," and ","Testing size: ",len(test_sent))
print(training_sent[0])

Training Size:  20000  and  Testing size:  6709
former versace store clerk sues over secret 'black code' for minority shoppers


In [42]:
# tokenizing and padding testing and training data

token=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
token.fit_on_texts(training_sent)
wi=token.word_index
training_seq=token.texts_to_sequences(training_sent)
print(training_sent[0],training_seq[0])

token.fit_on_texts(test_sent)
training_pad=pad_sequences(training_seq,maxlen=max_length,padding='post',truncating='post')

test_seq=token.texts_to_sequences(test_sent)
test_pad=pad_sequences(test_seq,maxlen=max_length,truncating='post')


former versace store clerk sues over secret 'black code' for minority shoppers [328, 1, 799, 3405, 2404, 47, 389, 2214, 1, 6, 2614, 8863]


In [85]:
#embedding is the means by which our model understand the sentiments of the word which are quite obvious for us through treaating each word as a vector

model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
     tf.keras.layers.Dropout(0.5),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(24,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



In [86]:
import numpy as np
training_pad=np.array(training_pad)

training_label=np.array(training_label)
test_pad=np.array(test_pad)
test_label=np.array(test_label)

In [87]:
hist=model.fit(training_pad,training_label,epochs=30,validation_data=(test_pad,test_label),verbose=2)

Epoch 1/30
625/625 - 2s - loss: 0.6389 - accuracy: 0.6288 - val_loss: 0.5879 - val_accuracy: 0.6727
Epoch 2/30
625/625 - 3s - loss: 0.4164 - accuracy: 0.8237 - val_loss: 0.6665 - val_accuracy: 0.6684
Epoch 3/30
625/625 - 3s - loss: 0.3438 - accuracy: 0.8566 - val_loss: 0.7332 - val_accuracy: 0.6493
Epoch 4/30
625/625 - 2s - loss: 0.2962 - accuracy: 0.8798 - val_loss: 0.8129 - val_accuracy: 0.6378
Epoch 5/30
625/625 - 2s - loss: 0.2744 - accuracy: 0.8875 - val_loss: 0.8800 - val_accuracy: 0.6278
Epoch 6/30
625/625 - 2s - loss: 0.2535 - accuracy: 0.8984 - val_loss: 0.9392 - val_accuracy: 0.6230
Epoch 7/30
625/625 - 2s - loss: 0.2420 - accuracy: 0.9052 - val_loss: 0.9710 - val_accuracy: 0.6221
Epoch 8/30
625/625 - 2s - loss: 0.2281 - accuracy: 0.9084 - val_loss: 1.0183 - val_accuracy: 0.6148
Epoch 9/30
625/625 - 2s - loss: 0.2172 - accuracy: 0.9162 - val_loss: 1.0597 - val_accuracy: 0.6110
Epoch 10/30
625/625 - 2s - loss: 0.2093 - accuracy: 0.9143 - val_loss: 1.0931 - val_accuracy: 0.6122

In [108]:
def predict(s):
  token.fit_on_texts(s)
  sq=token.texts_to_sequences(s)
  sp=pad_sequences(sq,maxlen=32,padding='post',truncating='post')

  prob=model.predict(sp)
  
  if(prob>0.6):
    print("Given sentence is sarcastic Don't die out of it")
  else:
    print('Take it seriously duh')





In [123]:
sen=['cows lose thier job as milk prices drop']
predict(sen)

[[0.7294824]]
Given sentence is sarcastic Don't die out of it
